In [1]:
import cv2, numpy as np
from mediapipe import solutions

In [ ]:
cap = cv2.VideoCapture(0)
WIDTH, HEIGHT, CHANNELS = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT), 3
mp_face_mesh = solutions.face_mesh
draw = solutions.drawing_utils
drawing_spec = draw.DrawingSpec(color=(0, 255, 0), circle_radius=1, thickness=1)

In [ ]:
with mp_face_mesh.FaceMesh() as face_mesh:
    while True:
        ok, frame = cap.read()
        if not ok:
            continue
        op = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) # modelo treinado com imagens no formato RGB
        if op.multi_face_landmarks:
            face_landmarks = op.multi_face_landmarks[0].landmark # objeto com todos os "landmarks" (pontos) do rosto
            
            ponto = draw._normalized_to_pixel_coordinates(face_landmarks[1].x, face_landmarks[1].y, WIDTH, HEIGHT) # Imagem com os pontos https://storage.googleapis.com/mediapipe-assets/documentation/mediapipe_face_landmark_fullsize.png

            # cv2.rectangle(frame, ponto - np.array([5, 5]), ponto + np.array([5, 5]), color=(0, 255, 0)) # desenha um quadrado na posição da variável ponto

            for landmarks in op.multi_face_landmarks: # op.multi_face_landmarks é uma lista que contém n listas de pontos (para n pessoas detectadas)
                draw.draw_landmarks(frame, landmarks, mp_face_mesh.FACEMESH_CONTOURS, landmark_drawing_spec=drawing_spec) # desenha os pontos no frame

        cv2.imshow("Video", frame)
        if cv2.waitKey(1) == 27:
            cv2.destroyAllWindows()
            break

In [ ]:
# Estima os ângulos nos eixos de rotação x, y e z
# adaptado de https://github.com/niconielsen32/ComputerVision/blob/master/headPoseEstimation.py

with mp_face_mesh.FaceMesh() as face_mesh:
    while True:
        ok, frame = cap.read()
        if not ok:
            continue
        op = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) # modelo treinado com imagens no formato RGB
        face_2d, face_3d = [], []
        if op.multi_face_landmarks:
            for landmarks in op.multi_face_landmarks: # op.multi_face_landmarks é uma lista que contém n listas de pontos (para n pessoas detectadas)
                for id, landmark in enumerate(landmarks.landmark):
                    if id in (1, 33, 61, 199, 263, 291): # 1: ponta do nariz; 33: canto esquerdo do olho esquerdo; 61: canto esquerdo da boca; 199: meio do queixo; 263: canto direito do olho direito; 291: canto direito da boca
                        if id == 1:
                            nose_2d = landmark.x*WIDTH, landmark.y*HEIGHT
                            nose_3d = landmark.x*WIDTH, landmark.y*HEIGHT, landmark.z*WIDTH
                        x, y = int(landmark.x*WIDTH), int(landmark.y*HEIGHT)

                        face_2d.append((x, y))
                        face_3d.append((x, y, landmark.z))
                
                face_2d = np.array(face_2d, dtype=np.float64)
                face_3d = np.array(face_3d, dtype=np.float64)

                focal_width = WIDTH
                camera_matrix = np.array([[focal_width, 0, HEIGHT/2],
                                          [0, focal_width, HEIGHT/2],
                                          [0, 0, 1]])
                distortion_matrix = np.zeros((4, 1), np.float64)
                success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, camera_matrix, distortion_matrix)
                rotation_matrix, jacobian = cv2.Rodrigues(rot_vec)
                angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rotation_matrix)
                x = angles[0]*360
                y = angles[1]*360
                z = angles[2]*360
                
                cv2.putText(frame, "x: " + str(np.round(x,2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(frame, "y: " + str(np.round(y,2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(frame, "z: " + str(np.round(z,2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                draw.draw_landmarks(frame, landmarks, mp_face_mesh.FACEMESH_CONTOURS, landmark_drawing_spec=drawing_spec)

        cv2.imshow("Video", frame)
        if cv2.waitKey(1) == 27:
            cv2.destroyAllWindows()
            break